In [1]:
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt
import seaborn as sns
import pandas as pd
import torch
import gymnasium as gym
import gym_bart
import itertools
from umap import UMAP
from plotting_utils import (
    set_rc, 
    add_abc_to_subaxes, 
    colors, 
    rgb_colors
)
from model_evaluation import (
    forced_action_evaluate, 
    meta_bart_callback,
    meta_bart_multi_callback,
    reshape_parallel_evalu_res,
    forced_action_evaluate_multi,
)
from bart_behavior_analysis import (
    plot_1color5fsize,
    plot_1colornfsize
)
from read_experiments import average_runs, load_exp_df
import re

from functools import partial
import sys
sys.path.append('../')
from ppo.envs import make_vec_env
import pickle
from tqdm import tqdm
from collections import defaultdict

from bart_representation_analysis import *
from bart_compress_visualize_decode import *
from sklearn.decomposition import PCA

set_rc()

In [2]:
size = np.arange(0.2, 1.01, 0.05)
env_kwargs = [{'meta_setup': 1, 'colors_used': 1, 
                            'max_steps': 2500, 'num_balloons': 50,
                            'inflate_noise': 0,
                            'fix_sizes': [0, s, 0]} for s in size]

giverew_env_kwargs = [{'meta_setup': 1, 'colors_used': 1, 
                            'max_steps': 2500, 'num_balloons': 50,
                            'inflate_noise': 0, 'give_rew': True,
                            'fix_sizes': [0, s, 0]} for s in size]

evalu = partial(forced_action_evaluate_multi, data_callback=meta_bart_multi_callback,
                env_name="BartMetaEnv", num_episodes=1, 
                env_kwargs=env_kwargs, 
                num_processes=17,
                seed=1,
                deterministic=False,
                with_activations=True)

giverew_evalu = partial(forced_action_evaluate_multi, data_callback=meta_bart_multi_callback,
                env_name="BartMetaEnv", num_episodes=1, 
                env_kwargs=giverew_env_kwargs, 
                num_processes=17,
                seed=1,
                deterministic=False,
                with_activations=True)

In [ ]:
r = pickle.load(open('data/meta_representation_results', 'rb'))

last_sizes = r['last_sizes']
unpopped_sizes = r['unpopped_sizes']
pop_rate = r['pop_rate']
rewards = r['rewards']
values = r['values']
action_probs = r['action_probs']
all_lens = r['all_lens']
all_num_balloons = r['all_num_balloons']

dec_flow_scores = r['dec_flow_scores']

iterators_idxs = r['iterators_idxs']
sizes = r['sizes']
ramp_f1s = r['ramp_f1s']
ramp_indiv_contribs = r['ramp_indiv_contribs']
confidence_scores = r['confidence_scores']
unconfidence_scores = r['unconfidence_scores']
unconfident_points = r['unconfident_points']
step_count = r['step_count']
all_decision_nodes = r['all_decision_nodes']

cluster_regressor_coefs = r['cluster_regressor_coefs']
cluster_regressor_scores = r['cluster_regressor_scores']
cluster_ks = r['cluster_ks']
pca_regressor_coefs = r['pca_regressor_coefs']
pca_regressor_scores = r['pca_regressor_scores']

In [11]:
"""
Main section to collect a bunch of data on confidence, activations
and ramping signal
"""
import warnings
warnings.filterwarnings("ignore")

give_rew = ['', 'giverew_']
postfixes = ['', 'pop0.05', 'pop0.1', 'pop0.2']
models = [1.0, 1.2, 1.5, 1.7, 2.0]
trials = range(3)
chks = np.arange(10, 243, 30)

iterators = [give_rew, postfixes, models, trials, chks]
iterators_idxs = [range(len(i)) for i in iterators]
sizes = [len(i) for i in iterators]

# indexed by model, then layer (shared0/1, actor0/1, critic0/1), finally index
#  first is fit to all activations, then fit to PCA i-1
# Performance
last_sizes = np.zeros(sizes + [17, 50])
unpopped_sizes = np.zeros(sizes + [17, 50])
pop_rate = np.zeros(sizes + [17])
rewards = np.zeros(sizes + [17, 2500])
values = np.zeros(sizes + [17, 2500])
action_probs = np.zeros(sizes + [17, 2500])
all_lens = np.zeros(sizes + [17])
all_num_balloons = np.zeros(sizes + [17])
balloon_steps = np.full(sizes + [17, 50], -1)
button_presses = np.full(sizes + [17, 100], -1)

# Decision flow
dec_flow_scores = np.zeros(sizes + [2]) # last axis: 0=decision nodes, 1=non-dec nodes

# Ramp to threshold F1 scores
ramp_f1s = np.zeros(sizes + [6, 11]) 
ramp_indiv_contribs = np.zeros(sizes + [6, 64])

# Confidence scores
confidence_scores = np.zeros(sizes)
unconfidence_scores = np.zeros(sizes)
unconfident_points = np.zeros(sizes)
step_count = np.zeros(sizes)
all_decision_nodes = np.zeros(sizes + [17, 64])

# Cluster and PCA regressor coefs
cluster_regressor_coefs = np.zeros(sizes + [6, 3, 64])
cluster_regressor_scores = np.zeros(sizes + [6, 3])
cluster_ks = np.zeros(sizes + [6])
pca_regressor_coefs = np.zeros(sizes + [6, 3, 6])
pca_regressor_scores = np.zeros(sizes + [6, 3])

for h, i, j, k, l in tqdm(itertools.product(*iterators_idxs), total=np.prod(sizes)):
    if step_count[h, i, j, k, l] != 0:
        continue
    
    give = give_rew[h]
    postfix = postfixes[i]
    model = models[j]
    t = k
    chk = chks[l]
    
    if h == 1 and postfix == '':
        postfix = 'pop0'
    exp_name = f"{give}p{model}n50{postfix}"
    model, (obs_rms, ret_rms) = \
        torch.load(f'../saved_checkpoints/meta_v2/{exp_name}_{t}/{chk}.pt')
    
    if h == 1:
        res = giverew_evalu(model, obs_rms)
    else:
        res = evalu(model, obs_rms)
    res = reshape_parallel_evalu_res(res, meta_balloons=50)

    # Performance
    lens = np.array([len(d) for d in res['dones']])
    num_balloons = np.array([len(d) for d in res['data']['balloon_step']])
    unpop_size = get_sizes(res, obs_rms, last_only=True)
    for ep in range(17):
        last_sizes[h, i, j, k, l, ep, :num_balloons[ep]] = res['data']['last_size'][ep]
        unpopped_sizes[h, i, j, k, l, ep, :num_balloons[ep]] = unpop_size[ep]
        pop_rate[h, i, j, k, l, ep] = np.sum(res['data']['popped'][ep]) / num_balloons[ep]
        values[h, i, j, k, l, ep, :lens[ep]] = res['values'][ep].reshape(-1)
        rewards[h, i, j, k, l, ep, :lens[ep]] = res['rewards'][ep]
        action_probs[h, i, j, k, l, ep, :lens[ep]] = res['action_probs'][ep][:, 1]
        
        presses = np.argwhere(res['actions'][ep].reshape(-1) == 1).reshape(-1)
        bsteps = res['data']['balloon_step'][ep]
        button_presses[h, i, j, k, l, ep, :len(presses)] = presses
        balloon_steps[h, i, j, k, l, ep, :len(bsteps)] = bsteps
        
    all_lens[h, i, j, k, l] = lens
    all_num_balloons[h, i, j, k, l] = num_balloons
    
    # Decision flow
    try:
        score, dec_nodes = score_decision_flow(res, model)
        dec_flow_scores[h, i, j, k, l] = score.mean(axis=0)
        all_decision_nodes[h, i, j, k, l] = dec_nodes
    except:
        pass
    
    # Confidence scores    
    non_presses = (np.vstack(res['actions']) == 0).reshape(-1)
    presses = (np.vstack(res['actions']) == 1).reshape(-1)
    aps = np.vstack(res['action_probs'])[:, 1]
    confidence_scores[h, i, j, k, l] = aps[presses].mean()
    unconfidence_scores[h, i, j, k, l] = aps[non_presses].mean()
    unconfident_points[h, i, j, k, l] = ((aps > 0.2) & (aps < 0.8)).sum() 
    step_count[h, i, j, k, l] = len(aps)
    
    # Ramp to threshold scores
    f1_scores, individual_scores = score_logistic_classifiers(res)
    ramp_f1s[h, i, j, k, l] = f1_scores
    ramp_indiv_contribs[h, i, j, k, l] = individual_scores
    
    # Cluster and PCA regressor scores
    layers = ['shared0', 'shared1', 'actor0', 'actor1', 'critic0', 'critic1']
    for z, layer in enumerate(layers):
        coefs, scores = compute_regressor_coefficients(res, by_clusters=True, layer=layer)
        n_clusters = coefs.shape[1]
        cluster_regressor_coefs[h, i, j, k, l, z, :, :n_clusters] = coefs
        cluster_regressor_scores[h, i, j, k, l, z] = scores
        cluster_ks[h, i, j, k, l, z] = n_clusters

        coefs, scores = compute_regressor_coefficients(res, by_clusters=False, layer=layer)
        pca_regressor_coefs[h, i, j, k, l, z] = coefs
        pca_regressor_scores[h, i, j, k, l, z] = scores
    
    
                
pickle.dump({
    'last_sizes': last_sizes, 
    'unpopped_sizes': unpopped_sizes, 
    'pop_rate': pop_rate, 
    'rewards': rewards, 
    'values': values, 
    'action_probs': action_probs,
    'all_lens': all_lens, 
    'all_num_balloons': all_num_balloons, 
    'balloon_steps': balloon_steps,
    'button_presses': button_presses,
    
    'dec_flow_scores': dec_flow_scores,
    
    'iterators_idxs': iterators_idxs,
    'sizes': sizes,
    'ramp_f1s': ramp_f1s,
    'ramp_indiv_contribs': ramp_indiv_contribs,
    'confidence_scores': confidence_scores,
    'unconfidence_scores': unconfidence_scores,
    'unconfident_points': unconfident_points,
    'step_count': step_count,
    'all_decision_nodes': all_decision_nodes,
    
    'cluster_regressor_coefs': cluster_regressor_coefs, 
    'cluster_regressor_scores': cluster_regressor_scores, 
    'cluster_ks': cluster_ks, 
    'pca_regressor_coefs': pca_regressor_coefs, 
    'pca_regressor_scores': pca_regressor_scores, 
}, open('data/meta_representation_results', 'wb'))

100%|██████████| 960/960 [4:42:35<00:00, 17.66s/it]  


In [21]:
"""
Main section to collect a bunch of data on confidence, activations
and ramping signal
"""
import warnings
warnings.filterwarnings("ignore")

give_rew = ['', 'giverew_']
postfixes = ['', 'pop0.05', 'pop0.1', 'pop0.2']
models = [1.0, 1.2, 1.5, 1.7, 2.0]
trials = range(3)
chks = np.arange(10, 243, 30)

iterators = [give_rew, postfixes, models, trials, chks]
iterators_idxs = [range(len(i)) for i in iterators]
sizes = [len(i) for i in iterators]

pop_rate = np.zeros(sizes + [17])

for h, i, j, k, l in tqdm(itertools.product(*iterators_idxs), total=np.prod(sizes)):
    give = give_rew[h]
    postfix = postfixes[i]
    model = models[j]
    t = k
    chk = chks[l]
    
    if h == 1 and postfix == '':
        postfix = 'pop0'
    exp_name = f"{give}p{model}n50{postfix}"
    model, (obs_rms, ret_rms) = \
        torch.load(f'../saved_checkpoints/meta_v2/{exp_name}_{t}/{chk}.pt')
    
    if h == 1:
        res = giverew_evalu(model, obs_rms)
    else:
        res = evalu(model, obs_rms)
    res = reshape_parallel_evalu_res(res, meta_balloons=50)

    # Performance
    lens = np.array([len(d) for d in res['dones']])
    num_balloons = np.array([len(d) for d in res['data']['balloon_step']])
    unpop_size = get_sizes(res, obs_rms, last_only=True)
    for ep in range(17):
        pop_rate[h, i, j, k, l, ep] = np.sum(res['data']['popped'][ep]) / num_balloons[ep]

100%|██████████| 960/960 [34:25<00:00,  2.15s/it]


In [13]:
r = pickle.load(open('data/meta_representation_results', 'rb'))
last_sizes = r['last_sizes'] 
unpopped_sizes = r['unpopped_sizes'] 
pop_rate = r['pop_rate'] 
rewards = r['rewards'] 
values = r['values'] 
action_probs = r['action_probs']
all_lens = r['all_lens'] 
all_num_balloons = r['all_num_balloons'] 
dec_flow_scores = r['dec_flow_scores']
iterators_idxs = r['iterators_idxs']
sizes = r['sizes']
ramp_f1s = r['ramp_f1s']
ramp_indiv_contribs = r['ramp_indiv_contribs']
confidence_scores = r['confidence_scores']
unconfidence_scores = r['unconfidence_scores']
unconfident_points = r['unconfident_points']
step_count = r['step_count']
all_decision_nodes = r['all_decision_nodes']
cluster_regressor_coefs = r['cluster_regressor_coefs'] 
cluster_regressor_scores = r['cluster_regressor_scores'] 
cluster_ks = r['cluster_ks'] 
pca_regressor_coefs = r['pca_regressor_coefs'] 
pca_regressor_scores = r['pca_regressor_scores'] 

In [29]:
pickle.dump({
    'last_sizes': last_sizes, 
    'unpopped_sizes': unpopped_sizes, 
    'pop_rate': pop_rate, 
    'rewards': rewards, 
    'values': values, 
    'action_probs': action_probs,
    'all_lens': all_lens, 
    'all_num_balloons': all_num_balloons, 
    
    'dec_flow_scores': dec_flow_scores,
    
    'iterators_idxs': iterators_idxs,
    'sizes': sizes,
    'ramp_f1s': ramp_f1s,
    'ramp_indiv_contribs': ramp_indiv_contribs,
    'confidence_scores': confidence_scores,
    'unconfidence_scores': unconfidence_scores,
    'unconfident_points': unconfident_points,
    'step_count': step_count,
    'all_decision_nodes': all_decision_nodes,
    
    'cluster_regressor_coefs': cluster_regressor_coefs, 
    'cluster_regressor_scores': cluster_regressor_scores, 
    'cluster_ks': cluster_ks, 
    'pca_regressor_coefs': pca_regressor_coefs, 
    'pca_regressor_scores': pca_regressor_scores, 
}, open('data/meta_representation_results', 'wb'))